# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [1]:
# Imports
import os
import json
from datetime import datetime
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [2]:
# Environment and clients
load_dotenv(override=True)
openai_api_key = os.getenv("OPENAI_API_KEY")
openrouter_api_key = os.getenv("OPENROUTER_API_KEY")
OPENROUTER_BASE_URL = "https://openrouter.ai/api/v1"
OLLAMA_URL = "http://localhost:11434/v1"

# OpenAI (default API) — uses OPENAI_API_KEY
openai = OpenAI(api_key=openai_api_key) if openai_api_key else None
# OpenRouter (OpenAI-compatible) — use OPENROUTER_API_KEY from https://openrouter.ai/keys
openrouter = OpenAI(base_url=OPENROUTER_BASE_URL, api_key=openrouter_api_key) if openrouter_api_key else None
ollama = OpenAI(api_key="ollama", base_url=OLLAMA_URL)

# Only include models whose client is available
MODELS = {}
if openai_api_key:
    MODELS["GPT-4.1-mini (OpenAI)"] = "gpt-4.1-mini"
if openrouter_api_key:
    # OpenRouter model IDs: https://openrouter.ai/docs#models
    MODELS["GPT-4o-mini (OpenRouter)"] = "openai/gpt-4o-mini"
MODELS["Llama 3.2 (Ollama)"] = "llama3.2"
DEFAULT_MODEL_KEY = list(MODELS.keys())[0]

In [3]:
# System prompt: adds technical expertise (Week 1 style)
SYSTEM_PROMPT = """You are a helpful technical tutor who answers questions about Python, software engineering, data science, and LLMs.
Give clear, accurate explanations. If you need the current date to answer (e.g. "when was X released"), use the get_current_date tool.
Keep answers focused and use examples when helpful."""

In [4]:
# Tool: get current date (bonus tool for the assistant)
def get_current_date():
    """Return today's date in ISO format for the assistant to use when answering questions about dates."""
    return datetime.now().strftime("%Y-%m-%d")

# Tool definition for OpenAI-compatible API
get_current_date_tool = {
    "type": "function",
    "function": {
        "name": "get_current_date",
        "description": "Get today's date in YYYY-MM-DD format. Use when the user asks about 'today', 'current date', or when something was released.",
        "parameters": {"type": "object", "properties": {}, "additionalProperties": False},
    },
}
TOOLS = [get_current_date_tool]

In [5]:
def handle_tool_calls(message):
    """Execute tool calls and return tool response messages."""
    responses = []
    for tool_call in message.tool_calls:
        if tool_call.function.name == "get_current_date":
            result = get_current_date()
            responses.append({
                "role": "tool",
                "content": result,
                "tool_call_id": tool_call.id,
            })
    return responses

In [6]:
def get_client(model_key):
    """Return the right API client for the selected model."""
    if "OpenRouter" in model_key:
        return openrouter
    if "Ollama" in model_key or "llama" in model_key.lower():
        return ollama
    return openai

In [7]:
def chat_stream(message, history, model_key):
    """
    Chat with streaming. Uses system prompt for expertise, supports model switch, and tools (OpenAI only).
    History already includes the latest user message from the UI.
    """
    client = get_client(model_key)
    model_name = MODELS.get(model_key, list(MODELS.values())[0])
    use_tools = (client == openai and openai is not None) or (client == openrouter and openrouter is not None)
    tools_arg = TOOLS if use_tools else None

    # History already includes the new user message (added by user_submit)
    messages = [{"role": "system", "content": SYSTEM_PROMPT}]
    for h in history:
        messages.append({"role": h["role"], "content": h["content"]})
    new_history = list(history)
    assistant_content = ""

    if use_tools:
        while True:
            response = client.chat.completions.create(
                model=model_name, messages=messages, tools=tools_arg
            )
            msg = response.choices[0].message
            if getattr(msg, "finish_reason", None) == "tool_calls" and getattr(msg, "tool_calls", None):
                messages.append(msg)
                messages.extend(handle_tool_calls(msg))
                continue
            assistant_content = msg.content or ""
            break
        for i in range(1, len(assistant_content) + 1, 3):
            yield new_history + [{"role": "assistant", "content": assistant_content[:i]}]
        yield new_history + [{"role": "assistant", "content": assistant_content}]
    else:
        stream = client.chat.completions.create(
            model=model_name, messages=messages, stream=True
        )
        for chunk in stream:
            if chunk.choices and chunk.choices[0].delta.content:
                assistant_content += chunk.choices[0].delta.content
                yield new_history + [{"role": "assistant", "content": assistant_content}]
        if assistant_content:
            yield new_history + [{"role": "assistant", "content": assistant_content}]

In [8]:
# Gradio UI: model dropdown + chat with streaming
with gr.Blocks(title="Technical Q&A Assistant", theme=gr.themes.Soft()) as demo:
    gr.Markdown("## Technical Q&A Assistant (Week 2 Exercise)\nAsk questions about Python, software engineering, data science, or LLMs. Use the dropdown to switch models.")
    with gr.Row():
        model_dropdown = gr.Dropdown(
            choices=list(MODELS.keys()),
            value=DEFAULT_MODEL_KEY,
            label="Model",
        )
    chatbot = gr.Chatbot(type="messages", label="Chat", height=500)
    msg = gr.Textbox(label="Your question", placeholder="e.g. Explain list comprehensions in Python")
    submit_btn = gr.Button("Submit")

    def user_submit(message, history):
        if not message.strip():
            return history
        return history + [{"role": "user", "content": message}]

    msg.submit(
        user_submit,
        inputs=[msg, chatbot],
        outputs=[chatbot],
    ).then(
        chat_stream,
        inputs=[msg, chatbot, model_dropdown],
        outputs=[chatbot],
        queue=True,
    ).then(lambda: "", None, [msg])

    submit_btn.click(
        user_submit,
        inputs=[msg, chatbot],
        outputs=[chatbot],
    ).then(
        chat_stream,
        inputs=[msg, chatbot, model_dropdown],
        outputs=[chatbot],
        queue=True,
    ).then(lambda: "", None, [msg])

demo.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7871
* To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "c:\Users\USER\Andela\llm_engineering\.venv\Lib\site-packages\gradio\queueing.py", line 759, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\USER\Andela\llm_engineering\.venv\Lib\site-packages\gradio\route_utils.py", line 354, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\USER\Andela\llm_engineering\.venv\Lib\site-packages\gradio\blocks.py", line 2116, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\USER\Andela\llm_engineering\.venv\Lib\site-packages\gradio\blocks.py", line 1635, in call_function
    prediction = await utils.async_iteration(iterator)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\USER\Andela\llm_engineering\.venv\Lib\site-packages\gradio\utils.py", line 760, in async_